In [2]:
import pandas as pd
import numpy as np

# import matplotlib.pyplot as plt

## 读取数据

In [3]:
train = pd.read_csv(r'data/train.csv')
test = pd.read_csv(r'data/test.csv')
# data = pd.concat([train, test], axis=0)
# data

## 数据处理

### 缺失值

In [4]:
def missing_value(data):
	## 缺失代表没有的
	data['Fence'] = data['Fence'].fillna('Na')
	data['Alley'] = data['Alley'].fillna('Na')
	data['MiscFeature'] = data['MiscFeature'].fillna('Na')
	data['PoolQC'] = data['PoolQC'].fillna('Na')
	data['FireplaceQu'] = data['FireplaceQu'].fillna('Na')
	data['BsmtHalfBath'] = data['BsmtHalfBath'].fillna(0)
	data['BsmtFullBath'] = data['BsmtFullBath'].fillna(0)
	## 缺失填众数的
	data['Electrical'] = data['Electrical'].fillna(data['Electrical'].mode()[0])
	data['Functional'] = data['Functional'].fillna(data['Functional'].mode()[0])
	data['Exterior1st'] = data['Exterior1st'].fillna(data['Exterior1st'].mode()[0])
	data['Exterior2nd'] = data['Exterior2nd'].fillna(data['Exterior2nd'].mode()[0])
	data['KitchenQual'] = data['KitchenQual'].fillna(data['KitchenQual'].mode()[0])
	data['SaleType'] = data['SaleType'].fillna(data['SaleType'].mode()[0])
	data['Utilities'] = data['Utilities'].fillna(data['Utilities'].mode()[0])
	'''
	BsmtFinSF1, BsmtFinType1, BsmtFinSF2, BsmtFinType2, BsmtQual
	都是地下室，面积就一个缺失值，面积是0的要么是未完工要么是缺失值，缺失值默认没有地下室
	BsmtCond地下室状态，面积为零的是没有，有面积的填众数TA
	BsmtExposure，面积为零的是没有，有面积的是没花园
	'''
	data['BsmtUnfSF'] = data['BsmtUnfSF'].fillna(0)
	data['BsmtFinSF1'] = data['BsmtFinSF1'].fillna(0)
	data['BsmtFinSF2'] = data['BsmtFinSF2'].fillna(0)
	data['BsmtFinType1'] = data['BsmtFinType1'].fillna('Na')
	data['BsmtFinType2'] = data['BsmtFinType2'].fillna('Na')
	data['TotalBsmtSF'] = data['BsmtUnfSF'] + data['BsmtFinSF1'] + data['BsmtFinSF2']
	data['BsmtQual'] = data['BsmtQual'].fillna('na')
	data.loc[(data['BsmtCond'].isnull()) & (data['TotalBsmtSF']==0), 'BsmtCond'] = 'Na'
	data.loc[(data['BsmtCond'].isnull()) & (data['TotalBsmtSF']!=0), 'BsmtCond'] = 'TA'
	data.loc[(data['BsmtExposure'].isnull()) & (data['TotalBsmtSF']==0), 'BsmtExposure'] = 'Na'
	data.loc[(data['BsmtExposure'].isnull()) & (data['TotalBsmtSF']!=0), 'BsmtExposure'] = 'No'
	'''
	LotFrontage：Linear feet of street connected to property
	居住在一起的人LotFrontage是相似的，我们将邻居的LotFrontage填充进来
	MSZoning: Identifies the general zoning classification of the sale
	住宅类型，应该也是和邻居一样，不过就缺了四个
	'''
	data['LotFrontage'] = data.groupby('Neighborhood')['LotFrontage'].transform(lambda x:x.fillna(x.median()))
	data['MSZoning'] = data.groupby('Neighborhood')['MSZoning'].transform(lambda x:x.fillna(x.mode()[0]))
	'''
	GarageYrBlt：Year garage was built
	GarageArea：Size of garage in square feet
	GarageCars：Size of garage in car capacity
	分别表示车库的建造年份,面积,车位大小,这个值缺失很可能表示没有车库，用0填充；
	GarageYrBlt年份有个2207，应该是2007打错了，改回来
	车库的建造年份缺失值填最小值，年份越早越便宜，建的早比没有好（相关系数0.4）
	'''
	data.loc[data['GarageYrBlt']==2207, 'GarageYrBlt'] = 2007
	data['GarageYrBlt'] = data['GarageYrBlt'].fillna(data['GarageYrBlt'].min()-1)
	data['GarageArea'] = data['GarageArea'].fillna(0)
	data['GarageCars'] = data['GarageCars'].fillna(0)
	data['GarageType'] = data['GarageType'].fillna('Na')
	data.loc[(data['GarageCond'].isnull()) & (data['GarageArea']==0), 'GarageCond'] = 'Na'
	data.loc[(data['GarageCond'].isnull()) & (data['GarageArea']!=0), 'GarageCond'] = 'TA'
	data.loc[(data['GarageQual'].isnull()) & (data['GarageArea']==0), 'GarageQual'] = 'Na'
	data.loc[(data['GarageQual'].isnull()) & (data['GarageArea']!=0), 'GarageQual'] = 'TA'
	data.loc[(data['GarageFinish'].isnull()) & (data['GarageArea']==0), 'GarageFinish'] = 'Na'
	data.loc[(data['GarageFinish'].isnull()) & (data['GarageArea']!=0), 'GarageFinish'] = 'Unf'
	'''
	MasVnrArea,MasVnrType外墙类型和面积，面积缺失应该是没有，填0，对应类型无
	类型缺失有几个有面积的应该有东西，填众数
	'''
	data['MasVnrArea'] = data['MasVnrArea'].fillna(0)
	data.loc[(data['MasVnrType'].isnull()) & (data['MasVnrArea']==0), 'MasVnrType'] = 'None'
	data.loc[(data['MasVnrType'].isnull()) & (data['MasVnrArea']!=0), 'MasVnrType'] = 'BrkFace'
	return data

In [5]:
data = missing_value(train)

In [6]:
## 分类变量
cat_cols = list(data.select_dtypes(include = ['object']).columns) + ['MSSubClass', 'OverallQual', 'OverallCond', 'MoSold']
## 年份变量
year_cols = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']
## 连续变量
ctn_cols = list(set(data.columns) - set(cat_cols) - set(year_cols) - set(['Id', 'SalePrice']))

## 年份变量取相对值
year_df_list = []
for col in ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'YrSold']:
    year_df = data[col] - 2010
    year_df_list.append(year_df)
year_data = pd.concat(year_df_list, axis=1)
## 分类变量取哑变量
cat_df_list = []
for col in cat_cols:
    df = pd.get_dummies(data[col], prefix=col)
    cat_df_list.append(df)
cat_data = pd.concat(cat_df_list, axis=1)\
## 拼接
data = pd.concat([year_data, cat_data, data[ctn_cols]], axis=1)
data

,YearBuilt,YearRemodAdd,GarageYrBlt,YrSold,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,...,KitchenAbvGr,OpenPorchSF,TotalBsmtSF,MasVnrArea,EnclosedPorch,LotFrontage,GarageCars,LowQualFinSF,HalfBath,FullBath
0,-7,-7,-7.0,-2,False,False,False,True,False,False,...,1,61,856,196.0,0,65.0,2,0,1,2
1,-34,-34,-34.0,-3,False,False,False,True,False,False,...,1,0,1262,0.0,0,80.0,2,0,0,2
2,-9,-8,-9.0,-2,False,False,False,True,False,False,...,1,42,920,162.0,0,68.0,2,0,1,2
3,-95,-40,-12.0,-4,False,False,False,True,False,False,...,1,35,756,0.0,272,60.0,3,0,0,1
4,-10,-10,-10.0,-2,False,False,False,True,False,False,...,1,84,1145,350.0,0,84.0,3,0,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,-11,-10,-11.0,-3,False,False,False,True,False,False,...,1,40,953,0.0,0,62.0,2,0,1,2
1456,-32,-22,-32.0,0,False,False,False,True,False,False,...,1,0,1542,119.0,0,85.0,2,0,0,2
1457,-69,-4,-69.0,0,False,False,False,True,False,False,...,1,60,1152,0.0,0,66.0,1,0,0,2
1458,-60,-14,-60.0,0,False,False,False,True,False,False,...,1,0,1078,0.0,112,68.0,1,0,0,1


## 建模

In [8]:
from sklearn.model_selection import train_test_split
train_X,test_X,train_y,test_y = train_test_split(data,train['SalePrice'],train_size = 0.8)
#输出数据集大小
print ('原始数据集特征：',data.shape, 
       '\n训练数据集特征：',train_X.shape ,
      '\n测试数据集特征：',test_X.shape)

print ('\n原始数据集标签：',train['SalePrice'].shape, 
       '\n训练数据集标签：',train_y.shape ,
      '\n测试数据集标签：',test_y.shape)

原始数据集特征： (1460, 344) 
训练数据集特征： (1168, 344) 
测试数据集特征： (292, 344)

原始数据集标签： (1460,) 
训练数据集标签： (1168,) 
测试数据集标签： (292,)


## 线性

In [9]:
from sklearn.linear_model import LinearRegression
model_linear = LinearRegression()
model_linear.fit(train_X,train_y)
model_linear.predict(test_X)

array([ 282049.50632217,  219225.58188044,   79570.32927256,
        142012.44287327,  126114.78274364,  205723.72323102,
         78418.19480626,   96699.14263624,  162987.23240025,
        452282.18411153,  153091.80101974,  106207.85376602,
        352794.85523165,  125640.23383349,  278830.53715464,
        190656.56125463,  113610.42665985,  136364.54447454,
        141575.93169334,  152514.44958349,  217190.10237295,
        285322.64577331,  333862.88494753,  143914.71779763,
        207667.64368197,  204160.99144415,  136555.73746683,
        142906.83540804,  227465.02111987,  169610.14066672,
        287674.51735445,   84249.16399572,  137324.46109083,
        172662.33906352,  270241.24685015,  221229.07275889,
        283633.26957239,  139696.96648896,  175246.50895429,
        140396.00570913,  262971.53443562,  347114.17991107,
        250449.56568138,  106182.49668785,  307885.66524568,
        195710.51682785,  136389.9238819 ,  129142.84484414,
         88857.15822484,

In [11]:
test_y

753     275500
1239    265900
1412     90000
1005    149900
1370    105000
         ...  
647     155000
488     160000
1278    237000
136     143000
379     179000
Name: SalePrice, Length: 292, dtype: int64

## 评估

In [10]:
from sklearn.metrics import mean_squared_error

# 示例数据
predictions = model_linear.predict(test_X)
targets = test_y

mse = mean_squared_error(targets, predictions)
rmse = np.sqrt(mse)

print('均方根误差：', rmse)


均方根误差： 73257.57062445024
